# Classification: Klassische Methoden
In diesem Notebook versuchen wir die Klassifizierung in "Failure"/"No Failure " mit klassischen machine learning Methoden durchzuführen.

In [1]:
import pandas as pd
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn import metrics
import Preprocessing as pp
import Helpers
from keras.models import Sequential
from keras.layers import Dense

rootdir_train = 'data/train/'
rootdir_test = 'data/test/'

train_labels_path = 'data/train_label.csv'
test_labels_path = 'data/test_label.csv'

feature_path = 'data/features/'
feature_path_test = 'data/features_test/'

feature_path_red = 'data/features_reduced/'
feature_path_test_red = 'data/features_reduced_test/'

# Merge labels and data
train_labels = pd.read_csv(train_labels_path, index_col=0) #Don't use index numbers per row but CSV file name as index

In [2]:
drift = ['pitch motor 1 current', 'pitch motor 2 current', 'Pitch motor 3 current', 'x direction vibration value', 'y direction vibration value', 'hydraulic brake pressure', 'generator current', 'Inverter inlet temperature', 'inverter outlet temperature', 'inverter inlet pressure', 'inverter outlet pressure', 'wind tower ambient temperature', 'Wheel temperature', 'Wheel control cabinet temperature', 'Cabin temperature', 'Cabin control cabinet temperature', 'vane 1 pitch motor temperature', 'blade 2 pitch motor temperature', 'blade 3 pitch motor temperature', 'blade 1 inverter box temperature', 'blade 2 inverter box temperature', 'blade 3 inverter box temperature','inverter grid side current', 'Inverter grid side active power', 'inverter generator side power', 'generator operating frequency','generator stator temperature 1', 'generator stator temperature 2',  'generator stator temperature 3', 'generator stator temperature 4', 'Generator stator temperature 5', 'generator stator temperature 6', 'generator air temperature 1', 'generator air temperature 2','main bearing temperature 1', 'main bearing temperature 2', 'Pitch motor 1 power estimation', 'Pitch motor 2 power estimation', 'Pitch motor 3 power estimation', 'blade 1 battery box temperature', 'blade 2 battery box temperature', 'blade 3 battery box temperature','Inverter INU temperature', 'Inverter ISU temperature','atmospheric pressure', 'reactive power control status', 'reactive power set value', 'Inverter INU RMIO temperature','blade 1 angle','blade 2 angle','blade 3 angle','inverter grid side voltage','inverter grid side reactive power']
print(75-len(drift))

22


In [3]:
# Read feature data
data = pd.read_csv(feature_path + 'mean' + '.csv', index_col=0)
data.head()
data.drop(drift, axis = 1, inplace=True)
sample_data = data.sample(2500, random_state=0)
labels = data['label']
sample_labels = sample_data['label']
data.drop(['area','label'], axis = 1, inplace=True)
sample_data.drop(['area','label'], axis = 1, inplace=True)

In [4]:
# Test data
test_data = pd.read_csv(feature_path_test + 'mean' + '.csv', index_col=0)
test_label = test_data['label']
test_data.drop(['area','label'], axis = 1, inplace=True)

In [5]:
# ################################## Classificaton with basic methods ###############################
# Classification: SVM

#X_train, X_test, y_train, y_test = train_test_split(data[0:300], labels[0:300], test_size=0.2, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(sample_data, sample_labels, test_size=0.2, random_state=123)

In [6]:
#Create a svm Classifier
#clf = svm.SVC(kernel='linear', gamma='auto', verbose=1) # Linear Kernel
clf = BaggingClassifier(base_estimator=SVC(kernel='linear', gamma='auto'), n_estimators=10, random_state=0, n_jobs=-1)

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset (here still part of training dataset to avoid concept drift)
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred))



# Classification: Lineares Modell

# Classification: Logitisches Modell

# Classification: KNN

# Classification: Xgboost

# Classification: LightGBM

# Others ?

In [7]:
def calculateImportance(bm):
    importance = [model.coef_ for model in bm.estimators_]
    # print(importance)
    m_importance = np.mean(importance,axis=0)
    # print(m_importance)
    return np.squeeze(m_importance)

coeff = calculateImportance(clf)
mag = np.std(X_train, 0) * coeff
print(mag)

Accuracy: 0.574
F1: 0.5170068027210885
Wheel speed                               -2.394063
hub angle                                  0.419400
overspeed sensor speed detection value     0.025051
5 second yaw against wind average         -0.504094
Aircraft weather station wind speed        0.788445
wind direction absolute value             -0.158691
generator torque                           1.241425
generator power limit value               -1.004964
Rated hub speed                            1.002310
Fan current status value                   1.406525
hub current status value                  -0.181384
yaw state value                           -0.480667
yaw request value                          0.037831
blade 1 super capacitor voltage           -0.188724
blade 2 super capacitor voltage            0.514198
blade 3 super capacitor voltage            0.993728
drive 1 thyristor temperature             -1.575836
Drive 2 thyristor temperature             17.525093
Drive 3 thyristor tempera

In [9]:
thresh = 1
imp_cut = np.abs(mag.to_numpy()) > thresh
ind = np.squeeze(np.argwhere(np.abs(mag.to_numpy()) > thresh))
print(ind + 1)

[ 1  7  8  9 10 17 18]


In [10]:
data.iloc[:,ind]

,Wheel speed,generator torque,generator power limit value,Rated hub speed,Fan current status value,drive 1 thyristor temperature,Drive 2 thyristor temperature
0,0.736525,0.000000,2053.692377,16.716794,4.966368,0.000000,0.000000
1,10.632768,351.875000,2067.600000,16.830000,6.000000,0.000000,0.000000
2,7.365436,128.120805,2067.600000,16.830000,6.000000,0.000000,0.000000
3,0.000000,0.000000,2062.974497,16.792349,2.993289,0.000000,0.000000
4,-0.000068,0.000000,423.483784,10.900676,1.990991,0.000000,0.000000
...,...,...,...,...,...,...,...
33355,8.597455,217.553571,1786.484152,16.328393,5.986607,57.930804,571.116071
33356,8.587342,228.637975,2067.600000,16.830000,6.000000,55.860759,547.063291
33357,12.276050,477.803653,1730.937215,16.260982,5.972603,57.431507,567.511416
33358,6.767692,0.108173,755.036298,12.937043,5.971154,56.612981,559.038462


In [11]:
def classifyAfterMagnitudeEvaluation():
    clf = BaggingClassifier(base_estimator=SVC(kernel='linear', gamma='auto'), n_estimators=10, random_state=0, n_jobs=-1)

    #Train the model using the training sets
    clf.fit(data.iloc[:,ind], labels)

    #Predict the response for test dataset (here still part of training dataset to avoid concept drift)
    y_pred = clf.predict(test_data.iloc[:,ind])

    # Model Accuracy: how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(test_label, y_pred))
    print("F1:",metrics.f1_score(test_label, y_pred))

In [12]:
classifyAfterMagnitudeEvaluation()

In [16]:
from joblib import dump, load
import os.path

def save_model(model, name):
    path = os.path.join("models", name + ".model")
    dump(model, path)

def load_model(path):
    return load(path)

## SVC

Reduzierte Daten nach Data_Exploration_Importance_of_Features.ipynb

In [54]:
def prepare_ml(estimator, x, y):
    clf = BaggingClassifier(base_estimator=estimator, bootstrap=False, n_estimators=10, random_state=0, n_jobs=-1)
    clf.fit(x,y)
    return clf
    

In [ ]:
def prepare_svm(red_drift = True, k='linear'):
    svc = SVC(kernel=k, gamma='auto')
    x, y = prepare_data(feature_path_red, red_drift)
    svc = prepare_ml(svc, x, y)

    return svc

In [55]:
from sklearn.neighbors import KNeighborsClassifier

def prepare_knn(red_drift = True):
    knn = KNeighborsClassifier()
    x, y = prepare_data(feature_path_red, red_drift)
    svc = prepare_ml(knn, x, y)

    return knn

In [58]:
def prepare_data(path, red_drift = True):
    data = pd.read_csv(path + 'mean' + '.csv', index_col=0)
    data.dropna(inplace=True)
    if red_drift:
        drl = list(set(data.columns.values).intersection(drift))
        data.drop(drl, axis = 1, inplace=True)
    label = data['label']
    data.drop(['area','label'], axis = 1, inplace=True)
    return data, label

In [21]:
def score_predict(test_label, prediction):
    print("Accuracy:",metrics.accuracy_score(test_label, prediction))
    print("F1:",metrics.f1_score(test_label, prediction))

### Mit Drift

#### SVC

In [ ]:
single_svc = prepare_svm()
save_model(single_svc,"SVC_reducedData")

In [65]:
x_test, y_test = prepare_data(feature_path_test_red)
y_predict_red = single_svc.predict(x_test)
score_predict(y_test,y_predict_red)

Accuracy: 0.5084123380958739
F1: 0.3411185682326621


#### KNN

In [66]:
single_knn = prepare_knn()
save_model(single_knn,"KNN_reducedData")

In [67]:
y_predict_red = single_svc.predict(x_test)
score_predict(y_test,y_predict_red)

Accuracy: 0.5084123380958739
F1: 0.3411185682326621


### Erkenntnis

Sowohl die SVM als auch der KNN raten nur das Ergebnis.
Anscheinend sind die Daten aus den unterschiedlichen Standorten untereinander zu unterschiedlich um gemeinsame Parameter zu finden.

### Ohne Drift

#### SVC

In [ ]:
single_svc_wd = prepare_svm(False)
save_model(single_svc_wd, "SVC_reducedData_withDrift")

In [68]:
x_test, y_test = prepare_test_data(False)
y_predict_red = single_svc_wd.predict(x_test)
score_predict(y_test,y_predict_red)

Accuracy: 0.5075443984510616
F1: 0.4366025053467767


#### KNN

In [69]:
single_knn = prepare_knn(False)
save_model(single_knn,"KNN_reducedData_withDrift")

In [70]:
y_predict_red = single_svc.predict(x_test)
score_predict(y_test,y_predict_red)

ValueError: Number of features of the model must match the input. Model n_features is 15 and input n_features is 54.

In [ ]:
# ################################## Classificaton with deep learning ###############################

# Classification: Neural Network
# (currently separate file 'deepNN.py' instead of jupyter notebook, but should be here before submission)